In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam
from keras import backend as K
from keras.layers import Conv2D,MaxPooling2D,UpSampling2D,Input,BatchNormalization,LeakyReLU,Conv2DTranspose,Dense,Softmax
from keras.layers.merge import concatenate
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from tensorflow import set_random_seed
import tensorflow as tf
tf.get_logger().setLevel('ERROR') # Filter out annoying WARNING deprecated method logs

set_random_seed(123)
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
#global graph
graph = tf.get_default_graph()
sess = tf.Session(graph=graph, config=session_conf)

#gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.9)
#sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=8, inter_op_parallelism_threads=8, gpu_options=gpu_options))



tf.keras.backend.set_session(sess)
set_random_seed(2)
np.random.seed(1)

#print(os.listdir("input/dataset/dataset_updated/"))

Using TensorFlow backend.


In [ ]:
from keras import backend as K
import sklearn.neighbors as nn
import random
channel = 3
epsilon = 1e-8
T = 0.38
img_rows=256
img_cols =256
nb_neighbors = 5

h, w = img_rows // 4, img_cols // 4

In [ ]:
# Load the array of quantized ab value
q_ab = np.load("npy/pts_in_hull.npy")
nb_q = q_ab.shape[0]

# Fit a NN to q_ab
nn_finder = nn.NearestNeighbors(n_neighbors=nb_neighbors, algorithm='ball_tree').fit(q_ab)

In [ ]:
from tensorflow import keras
modelT = keras.models.load_model('final_models/keras_zhang_model.hdf5')

In [ ]:
image_folder_pred = 'data/'
img_names_pred=os.listdir(image_folder_pred)
pred_names = [f for f in img_names_pred if f.lower().endswith('.jpeg')]

names_file = 'prediction_names.txt'
with open(names_file, 'w') as file:
    file.write('\n'.join(pred_names))

with open(names_file, 'r') as f:
    names = f.read().splitlines()
print(len(names))
samples = random.sample(names, 10)

#https://stackoverflow.com/a/50941282
#with graph.as_default():
for i in range(len(samples)):
    image_name = samples[i]
    filename = os.path.join(image_folder_pred, image_name)
    print('filename', filename)
    # b: 0 <=b<=255, g: 0 <=g<=255, r: 0 <=r<=255.
    bgr = cv2.imread(filename)
    if bgr is None:
        continue

    print('Start processing image: {}'.format(filename))
    gray = cv2.imread(filename, 0)
    bgr = cv2.resize(bgr, (img_rows, img_cols), cv2.INTER_CUBIC)
    gray = cv2.resize(gray, (img_rows, img_cols), cv2.INTER_CUBIC)
    # L: 0 <=L<= 255, a: 42 <=a<= 226, b: 20 <=b<= 223.
    lab = cv2.cvtColor(bgr, cv2.COLOR_BGR2LAB)
    L = lab[:, :, 0]
    a = lab[:, :, 1]
    b = lab[:, :, 2]
    # print('np.max(L): ' + str(np.max(L)))
    # print('np.min(L): ' + str(np.min(L)))
    # print('np.max(a): ' + str(np.max(a)))
    # print('np.min(a): ' + str(np.min(a)))
    # print('np.max(b): ' + str(np.max(b)))
    # print('np.min(b): ' + str(np.min(b)))
    x_test = np.empty((1, img_rows, img_cols, 1), dtype=np.float32)
    x_test[0, :, :, 0] = gray / 255.

    # L: 0 <=L<= 255, a: 42 <=a<= 226, b: 20 <=b<= 223.
    X_colorized = modelT.predict(x_test)
    #K.clear_session()
    X_colorized = X_colorized.reshape((h * w, nb_q))

    # Reweight probas
    X_colorized = np.exp(np.log(X_colorized + epsilon) / T)
    X_colorized = X_colorized / np.sum(X_colorized, 1)[:, np.newaxis]

    # Reweighted
    q_a = q_ab[:, 0].reshape((1, 313))
    q_b = q_ab[:, 1].reshape((1, 313))

    X_a = np.sum(X_colorized * q_a, 1).reshape((h, w))
    X_b = np.sum(X_colorized * q_b, 1).reshape((h, w))
    # print('np.max(X_a): ' + str(np.max(X_a)))
    # print('np.min(X_a): ' + str(np.min(X_a)))
    # print('np.max(X_b): ' + str(np.max(X_b)))
    # print('np.min(X_b): ' + str(np.min(X_b)))
    X_a = cv2.resize(X_a, (img_rows, img_cols), cv2.INTER_CUBIC)
    X_b = cv2.resize(X_b, (img_rows, img_cols), cv2.INTER_CUBIC)

    # Before: -90 <=a<= 100, -110 <=b<= 110
    # After: 38 <=a<= 228, 18 <=b<= 238
    X_a = X_a + 128
    X_b = X_b + 128
    # print('np.max(X_a): ' + str(np.max(X_a)))
    # print('np.min(X_a): ' + str(np.min(X_a)))
    # print('np.max(X_b): ' + str(np.max(X_b)))
    # print('np.min(X_b): ' + str(np.min(X_b)))

    out_lab = np.zeros((img_rows, img_cols, 3), dtype=np.int32)
    out_lab[:, :, 0] = lab[:, :, 0]
    out_lab[:, :, 1] = X_a
    out_lab[:, :, 2] = X_b
    out_L = out_lab[:, :, 0]
    out_a = out_lab[:, :, 1]
    out_b = out_lab[:, :, 2]
    # print('np.max(out_L): ' + str(np.max(out_L)))
    # print('np.min(out_L): ' + str(np.min(out_L)))
    # print('np.max(out_a): ' + str(np.max(out_a)))
    # print('np.min(out_a): ' + str(np.min(out_a)))
    # print('np.max(out_b): ' + str(np.max(out_b)))
    # print('np.min(out_b): ' + str(np.min(out_b)))
    out_lab = out_lab.astype(np.uint8)
    out_bgr = cv2.cvtColor(out_lab, cv2.COLOR_LAB2BGR)
    # print('np.max(out_bgr): ' + str(np.max(out_bgr)))
    # print('np.min(out_bgr): ' + str(np.min(out_bgr)))
    out_bgr = out_bgr.astype(np.uint8)

    if not os.path.exists('images'):
        os.makedirs('images')

    cv2.imwrite('images/{}_image.png'.format(i), gray)
    cv2.imwrite('images/{}_gt.png'.format(i), bgr)
    cv2.imwrite('images/{}_out.png'.format(i), out_bgr)

K.clear_session()